<a href="https://colab.research.google.com/github/alanwuha/ce7455-nlp/blob/master/practice/SkipGram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skip Gram
- [Ref](https://blog.cambridgespark.com/tutorial-build-your-own-embedding-and-use-it-in-a-neural-network-e9cde4a81296?gi=1cb19fad9b7a)
- [Code](https://github.com/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/02.Skip-gram-Negative-Sampling.ipynb)
- Edited by Hancheol Moon

![skip-gram](https://cdn-images-1.medium.com/max/800/1*SR6l59udY05_bUICAjb6-w.png)

- Skip-gram's objective is to predict the contexts, given a target word: V_t -> V_c
- The contexts are immediate neighbours of the target and are retrieved using a window of an arbitrary size _n_
  - Capturing _n_ words to the left of the target and _n_ words to its right.
- In a two-gram example:

![two-gram](https://nbviewer.jupyter.org/github/DSKSD/DeepNLP-models-Pytorch/blob/master/images/01.skipgram-prepare-data.png)

- The original Skip-grams' objective is to maximize P(V_c|V_c): The probability of V_c being predicted as V_t's context for all training pairs.
- To calculate P(V_c|V_t) we need a way to quantify the __closeness__ of the target-word around the context-word.
- In Skip-gram, this closeness is computed using the __dot product between the input-embedding of the target and the output-embedding of the context__.

Now, if we define u_t,c to be the measure of words' closeness between the target word and context word, _E_ to be the embedding matrix holding input-embeddings and _O_ to be the output-embedding matrix we get:

u_t,c = E_t O_c

, where _c_ is the context and _t_ is the target. With softmax,

![architecture](https://cdn-images-1.medium.com/max/1600/1*4Viy_LvP6jLIWSvB9-Fk-Q.png)

# Negative Sampling

So far, we have studied the basics of Skip-gram, but there is an issue with the __original softmax objective of Skip-gram__. It is __highly computationally expensive__:
- It requires scanning through the output-embeddings of all words in the vocabulary in order to calculate the sum from the __denominator__.
- Typically such vocabularies contain hundreds of thousands of words.
Because of this inefficiency, most implementations use an alternative, _negative-sampling objective_, which rephrases the problem as a set of independent binary classification tasks.

Instead of defining the complete probability distribution over words, __the model learns to distinguish the correct training pairs from incorrect pairs, which are randomly generated pairs.__
- Negative pair: Keep V_t and sample V_c from noise distribution
- D: correct pairs
- D': all negatively sampled |D| x m pairs
- P(C = 1 | V_t, V_c): the probability of (V_t, V_t) being a correct pair

For each sample we are making a binary decision we define P(C = 1|V_t, V_c) using the sigmoid function.

Negative (context) samples are drawn from uniform distribution raised to the power of 3/4. Why? If you play with some sample values, you'll find that, compared to the simpler equation, this one has the tendency to increase the probability for less frequent words and decrease the probability for more frequent words.

P(w) = Unif(W)^3/4 / Z,

where Z is the normalization factor.

__Sampling-based approaches completely do away with the softmax layer.__ They do this by approximating the normalization in the denominator of the softmax with some other loss that is cheap to compute. __However, sampling-based approaches are only useful at training time - - during inference, the full softmax still needs to be computed to obtain a normalised probability.

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional

import nltk
import random
import numpy as np
from collections import Counter

import pdb

flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

In [0]:
def getBatch(batch_size, train_data):
  random.shuffle(train_data) # Shuffling is necessary. Why?
  sindex = 0
  eindex = batch_size
  while eindex < len(train_data):
    batch = train_data[sindex:eindex]
    temp = eindex
    eindex = eindex + batch_size
    sindex = temp
    yield batch

  if eindex >= len(train_data):
    batch = train_data[sindex:]
    yield batch

In [0]:
def prepare_sequence(seq, word2index):
  l = lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>']
  idxs = list(map(l, seq))
  return torch.Tensor(idxs).type(torch.LongTensor)

def prepare_word(word, word2index):
  return torch.Tensor([word2index[word]]).type(torch.LongTensor) if word2index.get(word) is not None else torch.LongTensor([word2index['<UNK>']])

In [4]:
nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [5]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [0]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:500] # sampling sentences for test
corpus = [[word.lower() for word in sent] for sent in corpus]

In [7]:
print(len(corpus))
print(corpus[0], len(corpus[0]))

500
['[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']'] 8


In [0]:
# Exclude sparse words
MIN_COUNT = 3
word_count = Counter(flatten(corpus))
exclude = []
for w, c in word_count.items():
  if c < MIN_COUNT:
    exclude.append(w)

In [9]:
vocab = list(set(flatten(corpus)) - set(exclude))
vocab.append('<UNK>')
len(vocab)

479

In [0]:
word2index = {'<UNK>': 0}

for vo in vocab:
  if word2index.get(vo) is None:
    word2index[vo] = len(word2index)

  index2word = {v:k for k, v in word2index.items()}

In [0]:
WINDOW_SIZE = 5 # Range of contexts
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE,
                                    WINDOW_SIZE * 2 + 1)) for c in corpus])

In [12]:
print(windows[0])
print(windows[1])

('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville')
('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville', '1851')


In [13]:
list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + corpus[0] + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1))

[('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville'),
 ('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851'),
 ('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']'),
 ('<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>'),
 ('<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>'),
 ('[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>'),
 ('moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>'),
 ('dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>')]

In [0]:
# Create Training Set
train_data = []
for window in windows:
  for i in range(WINDOW_SIZE * 2 + 1):
    if window[i] in exclude or window[WINDOW_SIZE] in exclude:
      continue  # min_count

    if i == WINDOW_SIZE or window[i] == '<DUMMY>':
      continue

    # window[WINDOW_SIZE] : target word
    # window[i]           : context word
    train_data.append((window[WINDOW_SIZE], window[i]))

In [15]:
print(windows[0])
for w in windows[0]:
  print(w in exclude)

('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville')
False
False
False
False
False
True
True
True
False
True
True


In [16]:
# 2-Gram dataset
train_data[:10]

[('(', 'supplied'),
 ('(', 'by'),
 ('(', 'a'),
 ('(', 'late'),
 ('supplied', '('),
 ('supplied', 'by'),
 ('supplied', 'a'),
 ('supplied', 'late'),
 ('by', '('),
 ('by', 'supplied')]

In [0]:
X_p = []
y_p = []

for tr in train_data:
  X_p.append(prepare_word(tr[0], word2index).view(1, -1))
  y_p.append(prepare_word(tr[1], word2index).view(1, -1))

train_data = list(zip(X_p, y_p))

In [18]:
train_data[:10]

[(tensor([[34]]), tensor([[3]])),
 (tensor([[34]]), tensor([[100]])),
 (tensor([[34]]), tensor([[64]])),
 (tensor([[34]]), tensor([[159]])),
 (tensor([[3]]), tensor([[34]])),
 (tensor([[3]]), tensor([[100]])),
 (tensor([[3]]), tensor([[64]])),
 (tensor([[3]]), tensor([[159]])),
 (tensor([[100]]), tensor([[34]])),
 (tensor([[100]]), tensor([[3]]))]

In [19]:
index2word[411]

'then'

In [20]:
len(train_data)

50242

### Unigram Distribution

In [0]:
word_count = Counter(flatten(corpus))
num_total_words = sum([c for w, c in word_count.items() if w not in exclude])

In [0]:
alpha = 3/4
noise_dist = {key: val / num_total_words ** alpha for key, val in word_count.items() if key not in exclude}
Z = sum(noise_dist.values())

noise_dist_normalized = {key: val / Z for key, val in noise_dist.items()}

In [23]:
print(f"Normalization factor Z:           {Z:.7}")
print(f'Noise distribution:               {noise_dist["by"]:.6}')
print(f'Normalized noise distribution:    {noise_dist_normalized["by"]:.6}')

Normalization factor Z:           9.397142
Noise distribution:               0.0566383
Normalized noise distribution:    0.00602719


In [24]:
import pandas as pd
from IPython.display import display

total = 100
alpha = 3/4
raised = total ** alpha

counts = pd.Series(list(range(0, 110, 10)))
dist = counts.apply(lambda x: x/total)
noise_dist = counts.apply(lambda x: x/raised)
norm_factor = sum(noise_dist)
noise_dist_norm = noise_dist.apply(lambda x: x/norm_factor)

df = pd.DataFrame({'counts': counts,
                   'dist': dist,
                   'noise_dist': noise_dist,
                   'noise_dist_norm': noise_dist_norm})

display(df)
print('total:', total)
print('raised:', raised)
print('norm_factor:', norm_factor)
print('sum', sum(noise_dist_norm))

,counts,dist,noise_dist,noise_dist_norm
0,0,0.0,0.000000,0.000000
1,10,0.1,0.316228,0.018182
2,20,0.2,0.632456,0.036364
3,30,0.3,0.948683,0.054545
4,40,0.4,1.264911,0.072727
5,50,0.5,1.581139,0.090909
6,60,0.6,1.897367,0.109091
7,70,0.7,2.213594,0.127273
8,80,0.8,2.529822,0.145455
9,90,0.9,2.846050,0.163636


total: 100
raised: 31.622776601683793
norm_factor: 17.392527130926087
sum 1.0


In [25]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [26]:
K = 10
np.random.choice(list(noise_dist_normalized.keys()), size=K, p=list(noise_dist_normalized.values()), replace=True)

array(['in', '.', 'over', 'of', 'from', 'no', 'the', '--', 'have', 'the'],
      dtype='<U11')

In [27]:
np.random.choice(['a', 'b', 'c', 'd'], size=4, p=[0.5, 0.5, 0, 0])

array(['b', 'b', 'a', 'b'], dtype='<U1')

### Simple Question: What is the difference between nn.Embedding() and nn.Linear()?
- Word embedding doesn't assume any bias. It assumes a zero-centered distribution
- Following codes represent the same network structure:
  - `nn.Linear(vocab_size, embed_dim, bias=False)`
  - `nn.Embedding(vocab_size, embed_dim)`
- However, `nn.Embedding()` is more efficient, why?
  - Input of `nn.Embedding()` is integer: index of one-hot-vector

In [0]:
def negative_sampling(targets, noise_dist_normalized, k):
  batch_size = targets.size(0)
  neg_samples = []
  for i in range(batch_size):
    nsample = []
    if device == 'cuda':
      # GPU -> CPU
      target_index = targets[i].data.cpu().tolist()[0]  # PyTorch Tensor -> List
    else:
      target_index = targets[i].data.tolist()[0]

    while len(nsample) < k: # num of sampling
      neg = np.random.choice(list(noise_dist_normalized.keys()),
                             size=1, p=list(noise_dist_normalized.values()))
      neg_word = neg[0]
      if word2index[neg_word] == target_index:
        continue
      nsample.append(neg_word)
    neg_samples.append(prepare_sequence(nsample, word2index).view(1, -1))

  return torch.cat(neg_samples)   

In [0]:
class SkipgramNegSampling(nn.Module):

  def __init__(self, vocab_size, embed_dim):
    super(SkipgramNegSampling, self).__init__()
    self.embedding_v = nn.Embedding(vocab_size, embed_dim)  # center embedding
    self.embedding_u = nn.Embedding(vocab_size, embed_dim)  # output embedding
    self.logsigmoid = nn.LogSigmoid()

    nn.init.xavier_normal_(self.embedding_v.weight)
    nn.init.xavier_normal_(self.embedding_u.weight)

  def forward(self, center_words, target_words, negative_words):
    center_embeds = self.embedding_v(center_words)
    target_embeds = self.embedding_u(target_words)

    neg_embeds = - self.embedding_u(negative_words)

    positive_score = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
    negative_score = torch.sum(neg_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2), 1).view(negs.size(0), -1)

    loss = self.logsigmoid(positive_score) + self.logsigmoid(negative_score)

    return -torch.mean(loss)

  def prediction(self, inputs):
    embeds = self.embedding_v(inputs)

    return embeds

In [0]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 30
NEG = 10 # Num of Negative Sampling

In [31]:
import torch
help(torch.bmm)

Help on built-in function bmm:

bmm(...)
    bmm(input, mat2, out=None) -> Tensor
    
    Performs a batch matrix-matrix product of matrices stored in :attr:`input`
    and :attr:`mat2`.
    
    :attr:`input` and :attr:`mat2` must be 3-D tensors each containing
    the same number of matrices.
    
    If :attr:`input` is a :math:`(b \times n \times m)` tensor, :attr:`mat2` is a
    :math:`(b \times m \times p)` tensor, :attr:`out` will be a
    :math:`(b \times n \times p)` tensor.
    
    .. math::
        \text{out}_i = \text{input}_i \mathbin{@} \text{mat2}_i
    
    .. note:: This function does not :ref:`broadcast <broadcasting-semantics>`.
              For broadcasting matrix products, see :func:`torch.matmul`.
    
    Args:
        input (Tensor): the first batch of matrices to be multiplied
        mat2 (Tensor): the second batch of matrices to be multiplied
        out (Tensor, optional): the output tensor.
    
    Example::
    
        >>> input = torch.randn(10, 3, 4

In [38]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

losses = []
model = SkipgramNegSampling(len(word2index), EMBEDDING_SIZE)
if device == 'cuda':
  model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

Device: cuda


In [39]:
model

SkipgramNegSampling(
  (embedding_v): Embedding(479, 30)
  (embedding_u): Embedding(479, 30)
  (logsigmoid): LogSigmoid()
)

In [0]:
def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']

In [0]:
for epoch in range(EPOCH):
  scheduler.step()
  for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
    inputs, targets = zip(*batch)

    inputs = torch.cat(inputs).to(device)
    targets = torch.cat(targets).to(device)
    negs = negative_sampling(targets, noise_dist_normalized, NEG).to(device)

    model.zero_grad()

    loss = model(inputs, targets, negs)

    loss.backward()
    optimizer.step()

    if i % 20 == 0:
      lr = get_lr(optimizer)
      print(f'Epoch: {epoch} || Iter: {i} || Loss: {loss:.2f} || LR: {lr:.6f}')

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch: 0 || Iter: 0 || Loss: 1.39 || LR: 0.001000
Epoch: 0 || Iter: 20 || Loss: 1.37 || LR: 0.001000
Epoch: 0 || Iter: 40 || Loss: 1.32 || LR: 0.001000
Epoch: 0 || Iter: 60 || Loss: 1.22 || LR: 0.001000
Epoch: 0 || Iter: 80 || Loss: 1.10 || LR: 0.001000
Epoch: 0 || Iter: 100 || Loss: 1.00 || LR: 0.001000
Epoch: 0 || Iter: 120 || Loss: 0.96 || LR: 0.001000
Epoch: 0 || Iter: 140 || Loss: 0.93 || LR: 0.001000
Epoch: 0 || Iter: 160 || Loss: 0.92 || LR: 0.001000
Epoch: 0 || Iter: 180 || Loss: 0.92 || LR: 0.001000
Epoch: 1 || Iter: 0 || Loss: 0.91 || LR: 0.001000
Epoch: 1 || Iter: 20 || Loss: 0.91 || LR: 0.001000
Epoch: 1 || Iter: 40 || Loss: 0.90 || LR: 0.001000
Epoch: 1 || Iter: 60 || Loss: 0.90 || LR: 0.001000
Epoch: 1 || Iter: 80 || Loss: 0.91 || LR: 0.001000
Epoch: 1 || Iter: 100 || Loss: 0.90 || LR: 0.001000
Epoch: 1 || Iter: 120 || Loss: 0.91 || LR: 0.001000
Epoch: 1 || Iter: 140 || Loss: 0.90 || LR: 0.001000
Epoch: 1 || Iter: 160 || Loss: 0.90 || LR: 0.001000
Epoch: 1 || Iter: 180 ||

In [0]:
def word_similarity(target, vocab):
  target_V = model.prediction(prepare_word(target, word2index).to(device))

  similarities = []
  for i in range(len(vocab)):
    if vocab[i] == target:
      continue

    vector = model.prediction(prepare_word(list(vocab)[i], word2index).to(device))

    cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0]
    similarities.append([vocab[i], cosine_sim])
  return sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

In [0]:
test = random.choice(list(vocab))
print(test)
word_similarity(test, vocab)